In [1]:
!pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 50.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install tensorflow

In [17]:
!pip install permetrics==1.3.0

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
# multivariate multi-step encoder-decoder lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM , GRU
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import math
import seaborn as sns
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from pandas.core.frame import DataFrame
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.optimizers import Adam
from keras.layers import Conv1D, Conv2D, MaxPooling2D, MaxPooling1D
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import MinMaxScaler
import csv
import collections
from scipy.stats import zscore
from datetime import datetime
import tensorflow as tf
import collections
from scipy.stats import zscore
from datetime import datetime
from permetrics.regression import RegressionMetric
import pickle
from keras.layers import Embedding, Bidirectional, SimpleRNN

In [5]:
print(keras.__version__)
print(tf.__version__)

3.10.0
2.19.0


## Required functions

### Company data prepration functions

In [6]:
# convert history into inputs and outputs
def to_supervised(X, y, n_input, n_output):
    Xs, ys = [], []
    for i in range(len(X) - n_input - n_output + 1):
        Xs.append(X.iloc[i:i+n_input].values)
        ys.append(y.iloc[i+n_input:i+n_input+n_output].values)
    return np.array(Xs), np.array(ys)

def get_last_val_set_features(data):
  # flatten data
  X = list()
  X.append(data.iloc[len(data)-5:, :])
  return X


In [7]:
#Test starting from 1-1-2017 based on 'Present Effictive short-term forecasts of Saudi stock price trends using technical indicators and large-scale multivariate time series'
# split a multivariate dataset into train/test sets
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame

def prepare_company_for_training_and_testing(company, selected_ti, target_col,
                                              n_input=5, n_output=5):

    # ===============================
    # 1. Split data temporally
    # ===============================
    train = company.loc[:'2015-12-31']
    val   = company.loc['2016-01-01':'2016-12-31']
    test  = company.loc['2017-01-01':]

    print(f'Train shape: {train.shape}, Val shape: {val.shape}, Test shape: {test.shape}')

    # ===============================
    # 2. Feature scaler (X)
    # ===============================
    feature_scaler = StandardScaler()
    feature_scaler.fit(train[selected_ti])

    train_x_scaled = DataFrame(
        feature_scaler.transform(train[selected_ti]),
        columns=selected_ti,
        index=train.index
    )

    val_x_scaled = DataFrame(
        feature_scaler.transform(val[selected_ti]),
        columns=selected_ti,
        index=val.index
    )

    test_x_scaled = DataFrame(
        feature_scaler.transform(test[selected_ti]),
        columns=selected_ti,
        index=test.index
    )

    # ===============================
    # 3. Target scaler (y)
    # ===============================
    target_scaler = StandardScaler()
    target_scaler.fit(train[[target_col]])

    train_y_scaled = DataFrame(
        target_scaler.transform(train[[target_col]]),
        columns=[target_col],
        index=train.index
    )

    val_y_scaled = DataFrame(
        target_scaler.transform(val[[target_col]]),
        columns=[target_col],
        index=val.index
    )

    test_y_scaled = DataFrame(
        target_scaler.transform(test[[target_col]]),
        columns=[target_col],
        index=test.index
    )

    print("*******************************************")
    print("Train data after multistep approach")

    # ===============================
    # 4. Supervised transformation
    # ===============================
    train_x, train_y = to_supervised(train_x_scaled, train_y_scaled,
                                     n_input, n_output)

    print("Validation multistep approach")
    val_x, val_y = to_supervised(val_x_scaled, val_y_scaled,
                                 n_input, n_output)

    print("Test multistep approach")
    test_x, test_y = to_supervised(test_x_scaled, test_y_scaled,
                                   n_input, n_output)

    return (
        train_x, train_y,
        val_x, val_y,
        test_x, test_y,
        test,
        feature_scaler,
        target_scaler
    )


### Company models creating and evaluation functions

In [38]:
def true_y_to_evaluation_form(y_true, n_input, n_out):
  y = list()
  for i in range(len(y_true) - n_input - n_out + 1):
      y.append(y_true.iloc[i+n_input:i+n_input+n_out].values)
  return np.array(y)

# make a forecast
def forecast(model, history):
	yhat = model.predict(history, verbose=0)
	# we only want the vector forecast
	return yhat

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted,scaler,target_scaler):
  predicted_not_normlized = target_scaler.inverse_transform(predicted)
  predicted_not_normlized_rounded = [np.round(num,2) for num in predicted_not_normlized]
  mse_score= mse(actual,predicted_not_normlized_rounded)
  rmse_score = sqrt(mse_score)
  mae_score = mae(actual,predicted_not_normlized_rounded)
  final_r2_score = r2_score(actual,predicted_not_normlized_rounded,multioutput='variance_weighted')
  evaluator = RegressionMetric(actual, predicted_not_normlized_rounded)
  mre_score = np.mean(evaluator.mean_relative_error())
  return mre_score, rmse_score, mae_score, final_r2_score, predicted_not_normlized_rounded

# summarize scores
def summarize_scores(name, mre_score, rmse_score,mae_score,final_r2_score):
  #s_scores = ', '.join(['%.1f' % s for s in scores])
  #print('%s: [%.3f] %s' % (name, score, s_scores))
  print(f'*************{name}**********')
  print("MAE: ",mae_score)
  print("MRE: ",mre_score)
  print("RMSE: ",rmse_score)
  print("R2: ",final_r2_score)

# evaluate a single model
def evaluate_model(test_x,test_y,model,scaler,target_scaler):
	# walk-forward validation over each week
  predictions = list()
  predictions = forecast(model, test_x)
  # evaluate predictions days for each week
  predictions = array(predictions)
  mse_score, rmse_score,mae_score,final_r2_score, y_hat_not_normlized = evaluate_forecasts(test_y, predictions,scaler,target_scaler)
  return mse_score, rmse_score,mae_score,final_r2_score,y_hat_not_normlized

In [9]:
def get_brnn_models_info(train_x, train_y, val_x, val_y,
                         test_x, test_y_for_evaluation,
                         company_name, evaluation_df,
                         predictions_df, scaler, target_scaler):

    verbose, epochs = 0, 32
    batch_size = math.floor(len(train_x) * 0.05)

    n_timesteps, n_features = train_x.shape[1], train_x.shape[2]

    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

    for i in [2,4,8,16,32,64,128,256,512,1024,
              3,9,27,81,243,729,
              100,200,300,400,500,600,700,800,900,1000]:

        model = Sequential()
        model.add(
            Bidirectional(
                SimpleRNN(i, activation='relu',
                          input_shape=(n_timesteps, n_features))
            )
        )
        model.add(Dense(5))
        model.compile(loss='mse', optimizer='adam')

        model.fit(
            train_x, train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x, val_y)
        )

        mre, rmse, mae_s, r2, preds = evaluate_model(
            test_x, test_y_for_evaluation,
            model, scaler, target_scaler
        )

        evaluation_df.loc[i, ['MAE_b','MRE_b','RMSE_b','R2_b']] = \
            mae_s, mre, rmse, r2

        # predictions_df[f"BRNN_{i}"] = preds.flatten()

        # pickle.dump(
        #     model,
        #     open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_BRNN_{i}', 'wb')
        # )

        summarize_scores('Proposed BRNN', mre, rmse, mae_s, r2)

    return evaluation_df, predictions_df


def get_gru_models_info(train_x, train_y, val_x, val_y,
                        test_x, test_y_for_evaluation,
                        company_name, evaluation_df,
                        predictions_df, scaler, target_scaler):

    verbose, epochs = 0, 32
    batch_size = math.floor(len(train_x) * 0.05)

    n_timesteps, n_features = train_x.shape[1], train_x.shape[2]
    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

    for i in [2,4,8,16,32,64,128,256,512,1024,
              3,9,27,81,243,729,
              100,200,300,400,500,600,700,800,900,1000]:

        model = Sequential()
        model.add(GRU(i, activation='relu',
                      input_shape=(n_timesteps, n_features)))
        model.add(Dense(5))
        model.compile(loss='mse', optimizer='adam')

        model.fit(
            train_x, train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x, val_y)
        )

        mre, rmse, mae_s, r2, preds = evaluate_model(
            test_x, test_y_for_evaluation,
            model, scaler, target_scaler
        )

        evaluation_df.loc[i, ['MAE_g','MRE_g','RMSE_g','R2_g']] = \
            mae_s, mre, rmse, r2

        # predictions_df[f"GRU_{i}"] = preds.flatten()

        # pickle.dump(
        #     model,
        #     open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_GRU_{i}', 'wb')
        # )

        summarize_scores('Proposed GRU', mre, rmse, mae_s, r2)

    return evaluation_df, predictions_df

def get_lstm_models_info(train_x, train_y, val_x, val_y,
                         test_x, test_y_for_evaluation,
                         company_name, evaluation_df,
                         predictions_df, scaler, target_scaler):

    verbose, epochs = 0, 32
    batch_size = math.floor(len(train_x) * 0.05)

    n_timesteps, n_features = train_x.shape[1], train_x.shape[2]
    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

    for i in [2,4,8,16,32,64,128,256,512,1024,
              3,9,27,81,243,729,
              100,200,300,400,500,600,700,800,900,1000]:

        model = Sequential()
        model.add(LSTM(i, activation='relu',
                       input_shape=(n_timesteps, n_features)))
        model.add(Dense(5))
        model.compile(loss='mse', optimizer='adam')

        model.fit(
            train_x, train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x, val_y)
        )

        mre, rmse, mae_s, r2, preds = evaluate_model(
            test_x, test_y_for_evaluation,
            model, scaler, target_scaler
        )

        evaluation_df.loc[i, ['MAE_l','MRE_l','RMSE_l','R2_l']] = \
            mae_s, mre, rmse, r2

        # predictions_df[f"LSTM_{i}"] = preds.flatten()

        # pickle.dump(
        #     model,
        #     open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_LSTM_{i}', 'wb')
        # )

        summarize_scores('Proposed LSTM', mre, rmse, mae_s, r2)

    return evaluation_df, predictions_df


In [20]:
def train_evalute_models_for_company(train_x, train_y, val_x, val_y,
                                     test_x, test,
                                     company_name, scaler, target_scaler):

    eval_cols = [
        'MAE_b','MRE_b','RMSE_b','R2_b',
        'MAE_g','MRE_g','RMSE_g','R2_g',
        'MAE_l','MRE_l','RMSE_l','R2_l'
    ]

    unit_indices = [
        2,4,8,16,32,64,128,256,512,1024,
        3,9,27,81,243,729,
        100,200,300,400,500,600,700,800,900,1000
    ]

    test_y_for_evaluation = true_y_to_evaluation_form(
        test['Close'], 5, 5
    )

    evaluation_df = pd.DataFrame(columns=eval_cols, index=unit_indices)

    predictions_df = DataFrame(
        data=test_y_for_evaluation.flatten(),
        columns=['Actual']
    )

    evaluation_df, predictions_df = get_lstm_models_info(
        train_x,train_y,val_x,val_y,
        test_x,test_y_for_evaluation,
        company_name,evaluation_df,predictions_df,
        scaler,target_scaler
    )

    evaluation_df, predictions_df = get_brnn_models_info(
        train_x,train_y,val_x,val_y,
        test_x,test_y_for_evaluation,
        company_name,evaluation_df,predictions_df,
        scaler,target_scaler
    )

    evaluation_df, predictions_df = get_gru_models_info(
        train_x,train_y,val_x,val_y,
        test_x,test_y_for_evaluation,
        company_name,evaluation_df,predictions_df,
        scaler,target_scaler
    )

    return evaluation_df, predictions_df


## Excution

In [40]:
def run_company_experiment_from_csv(csv_path, company_name):
    """
    Load company CSV from Drive, preprocess, train models, and evaluate.

    Parameters
    ----------
    csv_path : str
        Path to CSV file in Google Drive
    company_name : str
        Company ticker (e.g., 'AAPL')

    Returns
    -------
    evaluation_df : pd.DataFrame
    predictions_df : pd.DataFrame
    """

    # ===============================
    # 1. Load data
    # ===============================
    df = pd.read_csv(
        csv_path,
        index_col='Date',
        parse_dates=True
    )

    # ===============================
    # 2. Handle NaN / inf
    # ===============================
    df = df.replace([np.inf, -np.inf], 0).fillna(0)

    # ===============================
    # 3. Prepare supervised data
    # ===============================
    (
        train_x_normlized,
        train_y_normlized,
        val_x_normlized,
        val_y_normlized,
        test_x_normlized,
        test_y_normlized,
        test,
        scaler,
        target_scaler
    ) = prepare_company_for_training_and_testing(
        df,
        selected_ti=df.columns,
        target_col='Close'
    )

    # ===============================
    # 4. Train and evaluate models
    # ===============================
    evaluation_df, predictions_df = train_evalute_models_for_company(
        train_x_normlized,
        train_y_normlized,
        val_x_normlized,
        val_y_normlized,
        test_x_normlized,
        test,
        company_name,
        scaler,
        target_scaler
    )

    return evaluation_df, predictions_df


### AAPL

In [41]:
csv_path = "/content/drive/MyDrive/KFUPM/Graduation/data/US/AAPL_selected_features.csv"
company_name = "AAPL"
evaluation_df, predictions_df = run_company_experiment_from_csv(csv_path, company_name)

Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
Validation multistep approach
Test multistep approach


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


*************Proposed LSTM**********
MAE:  32.64016203912913
MRE:  0.6566179999999999
RMSE:  43.68101735137365
R2:  -14.516488045921072


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


*************Proposed LSTM**********
MAE:  32.61030204108109
MRE:  0.672326
RMSE:  34.44553434624675
R2:  -8.648810881081523


...

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


*************Proposed GRU**********
MAE:  1.60742352469914
MRE:  0.03218599999999999
RMSE:  2.1477340528907853
R2:  0.9624881098632962


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


*************Proposed GRU**********
MAE:  2.8007693229562647
MRE:  0.060770000000000005
RMSE:  3.2093681533464133
R2:  0.9162380037140333


(         MAE_b     MRE_b    RMSE_b      R2_b      MAE_g     MRE_g     RMSE_g  \
 2     4.053326    0.0825   5.01596  0.795395  33.346302  0.688594  35.189012   
 4     2.562451  0.051558  3.351342  0.908663   5.580552  0.113076   7.273881   
 8      1.30142  0.025846   1.85202  0.972107   1.161437  0.023738   1.578253   
 16     1.30265   0.02718  1.828852    0.9728   0.889139  0.018346   1.306202   
 32    1.232755  0.025206  1.763761  0.974702    1.43984  0.028416   2.345802   
 64    1.286344  0.026262  1.715296  0.976073   1.494172  0.028002   2.276487   
 128   2.174855  0.044466  2.598373  0.945095    1.68057   0.03254   2.326749   
 256   1.130528  0.024044  1.530372  0.980954   1.555265  0.030134   2.132682   
 512   1.455848   0.02975  1.813883  0.973244   1.436934  0.027662   1.989712   
 1024  0.990906  0.020762  1.360396   0.98495   3.704447  0.074624   4.349962   
 3     1.782814    0.0363  2.359036  0.954744   2.549629   0.05137   3.411581   
 9     2.628576  0.053426  3

In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/AAPL_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/AAPL_predictions.csv")

### BAC

In [ ]:
csv_path = "/content/drive/MyDrive/KFUPM/Graduation/data/US/BAC_selected_features.csv"
company_name = "BAC"
evaluation_df, predictions_df = run_company_experiment_from_csv(csv_path, company_name)

,Adjusted Close,Close,High,Low,Open,alma,decay,dema,ema,fwma,...,sma,swma,t3,tema,trima,vmwa,vwap,wcp,wma,zlma
2001-12-31,18.983713,31.475,31.875,31.410,31.875,0.0,31.475000,0.000,0.000,0.00,...,0.000,0.000000,0.000,0.000,0.000000,0.000000,31.586670,31.55875,0.000000,0.00000
2002-01-01,18.983713,31.475,31.875,31.410,31.875,0.0,31.475000,0.000,0.000,0.00,...,0.000,0.000000,0.000,0.000,0.000000,0.000000,31.586670,31.55875,0.000000,0.00000
2002-01-02,18.986729,31.480,31.550,31.055,31.155,0.0,31.480000,0.000,0.000,0.00,...,0.000,0.000000,0.000,0.000,0.000000,0.000000,31.361664,31.39125,0.000000,0.00000
2002-01-03,18.965622,31.445,31.950,31.380,31.480,0.0,31.445000,0.000,0.000,0.00,...,0.000,0.000000,0.000,0.000,0.000000,0.000000,31.591667,31.55500,0.000000,0.00000
2002-01-04,19.261154,31.935,32.075,31.505,31.560,0.0,31.934999,31.562,31.562,31.66,...,31.562,31.521111,31.562,31.562,31.521111,31.596972,31.838333,31.86250,31.621333,31.76333


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/BAC_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/BAC_predictions.csv")

### J&J

In [ ]:
csv_path = "/content/drive/MyDrive/KFUPM/Graduation/data/US/JNJ_selected_features.csv"
company_name = "JNJ"
evaluation_df, predictions_df = run_company_experiment_from_csv(csv_path, company_name)

,Adjusted Close,Close,High,Low,Open,alma,decay,dema,ema,fwma,...,sma,swma,t3,tema,trima,vmwa,vwap,wcp,wma,zlma
2001-12-31,33.781322,59.10,59.82,58.88,59.70,0.0,59.099998,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,59.266663,59.2250,0.000,0.000000
2002-01-01,33.781322,59.10,59.82,58.88,59.70,0.0,59.099998,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,59.266663,59.2250,0.000,0.000000
2002-01-02,33.552654,58.70,59.30,57.58,58.90,0.0,58.899998,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,58.526670,58.5700,0.000,0.000000
2002-01-03,33.541237,58.68,58.73,57.88,58.70,0.0,58.680000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,58.430008,58.4925,0.000,0.000000
2002-01-04,33.232563,58.14,58.90,57.95,58.68,0.0,58.480000,58.743996,58.743996,58.528333,...,58.743999,58.766666,58.743996,58.743996,58.766666,58.740904,58.330006,58.2825,58.588,58.046665


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/jnj_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/jnj_predictions.csv")

### GOOG

In [ ]:
csv_path = "/content/drive/MyDrive/KFUPM/Graduation/data/US/GOOG_selected_features.csv"
company_name = "GOOG"
evaluation_df, predictions_df = run_company_experiment_from_csv(csv_path, company_name)

,Adjusted Close,Close,High,Low,Open,alma,decay,dema,ema,fwma,...,sma,swma,t3,tema,trima,vmwa,vwap,wcp,wma,zlma
2001-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/goog_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/goog_predictions.csv")